In [1]:
import pandas as pd

# Neu geclustertes DataFrame laden
df = pd.read_excel("../../data/df_de_final_clustered.xlsx", engine="openpyxl")


In [3]:
# Überblick: Wie viele Fragen pro Cluster?
cluster_counts = df["Cluster_Final"].value_counts().sort_index()

print("Fragen pro Cluster:")
print(cluster_counts)

# Optional: Als DataFrame speichern
cluster_summary = cluster_counts.reset_index()
cluster_summary.columns = ["Cluster_Final", "Anzahl_Fragen"]


Fragen pro Cluster:
Cluster_Final
-1       1900
 0         59
 1         59
 2         60
 3          3
         ... 
 1310       3
 1311       3
 1312       2
 1313       3
 1314       2
Name: count, Length: 1316, dtype: int64


In [4]:
# Anzahl Fragen im Cluster -1 (= Noise)
noise_count = (df["Cluster_Final"] == -1).sum()
total_count = len(df)

print(f"Anteil Noise: {noise_count / total_count:.2%}")
print(f"Anteil geclusterte Fragen: {(1 - noise_count / total_count):.2%}")


Anteil Noise: 25.47%
Anteil geclusterte Fragen: 74.53%


In [5]:
# Top 10 grösste Cluster auswählen (ohne Noise)
top_clusters = (
    cluster_summary[cluster_summary["Cluster_Final"] != -1]  # Noise ausschliessen
    .sort_values(by="Anzahl_Fragen", ascending=False)
    .head(10)["Cluster_Final"]
    .tolist()
)

print(f"Top 10 Cluster: {top_clusters}")

# Funktion: Beispiele aus jedem der Top 10 Cluster anzeigen
def beispiele_je_top_cluster(df, cluster_ids, n=3):
    for cluster_id in cluster_ids:
        cluster_df = df[df["Cluster_Final"] == cluster_id]
        print(f"\n{'='*50}")
        print(f"Cluster {cluster_id} - {len(cluster_df)} Fragen")
        print(f"{'='*50}")
        
        beispiele = cluster_df["Frage_Text"].sample(n=min(n, len(cluster_df)), random_state=42)
        for idx, frage in enumerate(beispiele, 1):
            print(f"{idx}. {frage}")

# Ausführen
beispiele_je_top_cluster(df, top_clusters, n=3)


Top 10 Cluster: [5, 62, 60, 61, 129, 33, 2, 0, 22, 1]

Cluster 5 - 102 Fragen
1. Soll die Schweiz das Schengen-Abkommen mit der EU kündigen und wieder verstärkte Personenkontrollen direkt an der Grenze einführen?
2. Soll die Schweiz das Schengen-Abkommen mit der EU kündigen und wieder verstärkte Personenkontrollen direkt an der Grenze einführen?
3. Soll die Schweiz das Schengen-Abkommen mit der EU kündigen und wieder verstärkte Personenkontrollen direkt an der Grenze einführen?

Cluster 62 - 71 Fragen
1. Wie beurteilen Sie die folgende Aussage: "Ein stärkerer Umweltschutz ist notwendig, auch wenn er zu Lasten des Wirtschaftswachstums durchgesetzt werden muss."
2. Wie beurteilen Sie die folgende Aussage: "Ein stärkerer Umweltschutz ist notwendig, auch wenn er zu Lasten des Wirtschaftswachstums durchgesetzt werden muss."
3. Wie beurteilen Sie die folgende Aussage: "Ein stärkerer Umweltschutz ist notwendig, auch wenn er zu Lasten des Wirtschaftswachstums durchgesetzt werden muss."

Cluste

In [6]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows

# 1. DataFrame laden
df = pd.read_excel("../../data/df_de_final_clustered.xlsx", engine="openpyxl")

# 2. Cluster nach Grösse sortieren
cluster_counts = df["Cluster_Final"].value_counts().reset_index()
cluster_counts.columns = ["Cluster_Final", "Anzahl_Fragen"]

# 3. Neues Excel-Workbook erstellen
wb = Workbook()
ws = wb.active
ws.title = "Cluster-Auswertung"

# 4. Stil definieren für die Überschrift
header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")

# 5. Alle Cluster in Reihenfolge durchlaufen
for _, row in cluster_counts.iterrows():
    cluster_id = row["Cluster_Final"]
    cluster_df = df[df["Cluster_Final"] == cluster_id]

    # Prüfen, ob alle Fragen im Cluster identisch sind
    if cluster_df["Frage_Text"].nunique() == 1:
        hinweis = "alle Fragen identisch"
    else:
        hinweis = "unterschiedliche Fragen vorhanden"

    # Cluster-Überschrift erstellen
    ueberschrift = f"=== Cluster {cluster_id} – {len(cluster_df)} Fragen – {hinweis} ==="
    ws.append([ueberschrift])

    # Stil anwenden auf die Überschrift-Zeile
    for cell in ws[ws.max_row]:
        cell.font = header_font
        cell.fill = header_fill

    # Alle Datenzeilen des Clusters einfügen
    for r_idx, data_row in enumerate(dataframe_to_rows(cluster_df, index=False, header=True)):
        # Nur einmal die Header-Zeile ausgeben (bei jedem Cluster wieder)
        if r_idx == 0:
            ws.append(data_row)
        else:
            ws.append(data_row)

    # Leere Zeile als Trennung
    ws.append([])

# 6. Excel-Datei speichern
output_path = "../../data/df_clusters_auswertung_voll.xlsx"
wb.save(output_path)

print(f"Excel-Datei erfolgreich gespeichert: {output_path}")


Excel-Datei erfolgreich gespeichert: ../../data/df_clusters_auswertung_voll.xlsx


In [7]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows

# Geclustertes DataFrame laden
df = pd.read_excel("../../data/df_de_final_clustered.xlsx", engine="openpyxl")

# Filter: Cluster != -1 und Cluster mit mindestens 2 unterschiedlichen Fragen
valid_clusters = []

for cluster_id in df["Cluster_Final"].unique():
    if cluster_id == -1:
        continue  # Noise-Cluster ignorieren
    cluster_df = df[df["Cluster_Final"] == cluster_id]
    if cluster_df["Frage_Text"].nunique() >= 2:
        valid_clusters.append(cluster_id)

print(f"{len(valid_clusters)} gültige Cluster gefunden.")

# Neues Workbook erstellen
wb = Workbook()
ws = wb.active
ws.title = "Cluster-Auswertung"

# Definiere Stil für Cluster-Überschrift
header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")

# Alle Cluster durchgehen und einfügen
for cluster_id in valid_clusters:
    cluster_df = df[df["Cluster_Final"] == cluster_id]
    
    # Cluster-Überschrift einfügen
    ws.append([f"=== Cluster {cluster_id} – {len(cluster_df)} Fragen ==="])
    for cell in ws[ws.max_row]:
        cell.font = header_font
        cell.fill = header_fill

    # DataFrame-Daten einfügen
    for row in dataframe_to_rows(cluster_df, index=False, header=True):
        ws.append(row)

    # Leere Zeile zur visuellen Trennung
    ws.append([])

# Speichern
wb.save("../../data/df_clusters_auswertung_not_unique.xlsx")

print("Excel-Datei wurde erfolgreich erstellt: df_clusters_auswertung_not_unique.xlsx")


23 gültige Cluster gefunden.
Excel-Datei wurde erfolgreich erstellt: df_clusters_auswertung_not_unique.xlsx


In [8]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows

# DataFrame laden
df = pd.read_excel("../../data/df_de_final_clustered.xlsx", engine="openpyxl")

# Cluster nach Grösse sortieren
cluster_counts = df["Cluster_Final"].value_counts().reset_index()
cluster_counts.columns = ["Cluster_Final", "Anzahl_Fragen"]

# Neues Workbook
wb = Workbook()
ws = wb.active
ws.title = "Cluster-Auswertung"

# Stile definieren
header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")

standard_fill = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")  # Weiss
abweichung_fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")  # Hellgelb

# Statistikvariablen
gesamt_cluster = 0
cluster_mit_abweichung = 0

# Cluster durchlaufen
for _, row in cluster_counts.iterrows():
    cluster_id = row["Cluster_Final"]
    if cluster_id == -1:
        continue  # Noise ignorieren

    cluster_df = df[df["Cluster_Final"] == cluster_id]
    n_unique = cluster_df["Frage_Text"].nunique()  # ACHTUNG: hier OHNE Normierung!

    # Cluster-Statistiken zählen
    gesamt_cluster += 1
    if n_unique >= 2:
        cluster_mit_abweichung += 1

    hinweis = "unterschiedliche Fragen vorhanden" if n_unique >= 2 else "alle Fragen identisch"

    # Cluster-Überschrift einfügen
    ws.append([f"=== Cluster {cluster_id} – {len(cluster_df)} Fragen – {hinweis} ==="])
    for cell in ws[ws.max_row]:
        cell.font = header_font
        cell.fill = header_fill

    # Datenzeilen schreiben
    columns_to_export = df.columns.tolist()  # Alle Spalten exportieren
    ws.append(columns_to_export)

    häufigster_text = cluster_df["Frage_Text"].mode()[0]  # Häufigster Fragetext (original!)

    for _, zeile in cluster_df.iterrows():
        werte = [zeile[col] for col in columns_to_export]
        ws.append(werte)
        for idx, cell in enumerate(ws[ws.max_row]):
            if idx == columns_to_export.index("Frage_Text"):
                if zeile["Frage_Text"] == häufigster_text:
                    cell.fill = standard_fill
                else:
                    cell.fill = abweichung_fill

    ws.append([])

# Excel speichern
output_path = "../../data/df_clusters_auswertung_farbig_originalvergleich.xlsx"
wb.save(output_path)

# Zusammenfassung ausgeben
print(f"Excel-Datei erfolgreich gespeichert: {output_path}")
print(f"Anzahl Cluster in der Excel-Datei: {gesamt_cluster}")
print(f"Anzahl Cluster mit mindestens einer abweichenden Frage: {cluster_mit_abweichung}")


Excel-Datei erfolgreich gespeichert: ../../data/df_clusters_auswertung_farbig_originalvergleich.xlsx
Anzahl Cluster in der Excel-Datei: 1315
Anzahl Cluster mit mindestens einer abweichenden Frage: 23


In [10]:
frage_1 = df[df["ID_gesamt"] == "Q01330"]["Frage_Text"].iloc[0]
frage_2 = df[df["ID_gesamt"] == "Q01278"]["Frage_Text"].iloc[0]

# Vergleich: Sind sie exakt gleich?
print(f"Sind sie exakt gleich? {frage_1 == frage_2}")

# Längere Version für vollständigen Vergleich
max_len = max(len(frage_1), len(frage_2))

for i in range(max_len):
    char1 = frage_1[i] if i < len(frage_1) else "[Ende]"
    char2 = frage_2[i] if i < len(frage_2) else "[Ende]"
    if char1 != char2:
        print(f"Unterschied bei Zeichen {i}: '{char1}' vs. '{char2}' (Codepoints: {ord(char1) if char1 != '[Ende]' else 'None'} vs {ord(char2) if char2 != '[Ende]' else 'None'})")


Sind sie exakt gleich? False
Unterschied bei Zeichen 131: ' ' vs. '[Ende]' (Codepoints: 32 vs None)
